In [ ]:
import os
import subprocess

# Replace 'your_command' with the actual command you want to run
command_list = {
    "pwd"
}

# Expand the ~ character to the home directory
home_directory = os.path.expanduser('~')

# Path to the Git repository (using ~ in the path)
repository_path = [os.path.join(home_directory, 'Tools/apollo/'),0]

# Run the command and capture the output
for command in command_list:
    result = subprocess.Popen(command, cwd=repository_path[0], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
stdout, stderr = result.communicate()
# Print the output and return code
print("Output:", stdout)
print("Error:", stderr)
print("Return Code:", result.returncode)


In [ ]:
import os
import subprocess
import time

home_directory = os.path.expanduser('~')

# Commands to run
command_list = [
    {'command': "./CarlaUE4.sh", 'cwd': os.path.join(home_directory, 'Tools/CARLA_0.9.13/')},
    {'command': "./docker/scripts/dev_start.sh", 'cwd': os.path.join(home_directory, "Tools/apollo/")},
]

# Run the first command in a detached tmux session
tmux_command = ['tmux', 'new-session', '-d', '-s', 'carla_session', 'bash', '-c', command_list[0]['command']]
subprocess.run(tmux_command, cwd=command_list[0]['cwd'])

# Wait for a short time to ensure the first command starts running
time.sleep(2)

# Run the second command using subprocess without waiting for it to finish
subprocess.run(command_list[1]['command'], cwd=command_list[1]['cwd'])

# Optionally, you can attach to the tmux session for the first command if needed
# subprocess.run(['tmux', 'attach-session', '-t', 'carla_session'])


In [ ]:
import time
import os
import subprocess


# Expand the ~ character to the home directory
home_directory = os.path.expanduser('~')

command_list = [
    {'command': ['docker', 'exec', 'apollo_dev_weidong','shecho $PYTHONPATH ','-c',
            'cd', '/apollo/modules/carla_bridge/',
            '&&', 'chmod', '+x', 'install.sh',
            '&&', './install.sh',
            '&&', 'source', '~/.bashrc',
            '&&', 'python', 'main.py'
            ], 'cwd': home_directory},
]

# Run the command and capture the output
for command in command_list:
    print(f"Running command: {command['command']} in directory {command['cwd']}")
    result = subprocess.run(command['command'], cwd=command['cwd'], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Print the output and return code
    print("Output:", result.stdout)
    print("Error:", result.stderr)
    print("Return Code:", result.returncode)
    if result.returncode != 0:
        raise Exception(f"Command: {command['command']} failed. ")
    time.sleep(2)




In [ ]:
import time
import os
import subprocess

# Expand the ~ character to the home directory
home_directory = os.path.expanduser('~')

command_list = [
    {'command': ['docker', 'exec', '-w', '/apollo/modules/carla_bridge/', '-u', 'weidong', 'apollo_dev_weidong', 'sh', '-c',
                 'chmod +x install.sh && ./install.sh && source ~/.bashrc && tail ~/.bashrc && echo "python path: " $PYTHONPATH && python main.py'],
     'cwd': home_directory},
]

# Run the command and capture the output
for command in command_list:
    print(f"Running command: {' '.join(command['command'])} in directory {command['cwd']}")
    result = subprocess.run(command['command'], cwd=command['cwd'], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print(f"Command: {' '.join(command['command'])} returns code {result.returncode}. ")

    # Print the output and return code
    print("Output:", result.stdout)
    print("Error:", result.stderr)
    print("Return Code:", result.returncode)
    if result.returncode != 0:
        raise Exception(f"Command: {' '.join(command['command'])} failed. ")
    time.sleep(2)



In [ ]:
import time
import os
import subprocess


# Expand the ~ character to the home directory
home_directory = os.path.expanduser('~')
command_list = [
    {'command': ['./CarlaUE4.sh'], 'cwd': os.path.join(home_directory, 'Tools/CARLA_0.9.13/'), 'close_fds':True},
    {'command': ["./docker/scripts/dev_start.sh"], 'cwd': os.path.join(home_directory, "Tools/apollo/")},
    # {'command': ['docker','exec','-u','weidonghu','apollo_dev_weidonghu','tail','/home/weidonghu/.bashrc'], 'cwd': home_directory},
    # {'command': "./docker/scripts/dev_into.sh", 'cwd': os.path.join(home_directory, "Tools/apollo/")},
    {'command': ['git','checkout','--','modules/common/data/global_flagfile.txt'], 'cwd': os.path.join(home_directory, "Tools/apollo/")},
    {'command': ['docker','exec','-u','weidonghu','apollo_dev_weidonghu','./scripts/bootstrap.sh'], 'cwd': home_directory},
    {'command': ['docker','exec','-w','/apollo/modules/carla_bridge/','-u','weidonghu','apollo_dev_weidonghu', 'sh', '-c','source /home/weidonghu/.bashrc && tail /home/weidonghu/.bashrc && echo $PYTHONPATH && python main.py'], 'cwd': home_directory, 'close_fds':True},
    {'command': ['python3','./set_destination.py'], 'cwd': os.path.join(home_directory, "Tools/Scenic/scripts/")},
    # {'command': ['tmux', 'kill-session', '-t', 'carla_session'], 'cwd': home_directory}
]

subprocess.Popen(['./CarlaUE4.sh'], cwd=os.path.join(home_directory, 'Tools/CARLA_0.9.13/'), close_fds=True)

subprocess.Popen(["./docker/scripts/dev_start.sh"], cwd=os.path.join(home_directory, "Tools/apollo/"))


In [ ]:
import time
import os
import subprocess


# Expand the ~ character to the home directory
home_directory = os.path.expanduser('~')
subprocess.run(['tmux', 'kill-session', '-t', 'carla_session'])
# subprocess.run(['tmux', 'kill-session', '-t', 'bridge_session'])

In [ ]:
import time
import json
from pprint import pprint
from websocket import create_connection

ip = 'localhost'
port = '8888'
url = "ws://" + ip + ":" + port + "/websocket"
ws = create_connection(url)
status = None

while True:
    try:
        ws.close()
        ws = create_connection(url)
    except ConnectionRefusedError as e:
        print("connection refused.")
    data = json.loads(ws.recv())
    if data != status:
        status = data
        with open('status_info.json', 'a+') as f:
            json.dump(data, f, indent=4)


In [3]:
import docker

# Connect to the Docker daemon
client = docker.from_env()

def is_container_running(container_name):
    try:
        # Get container object
        container = client.containers.get(container_name)
        # Check if container is running
        return container.status == "running"
    except docker.errors.NotFound:
        # Container does not exist
        return False

# Example usage
container_name = "apollo_dev_weidong"
if is_container_running(container_name):
    print(f"Container {container_name} is running.")
else:
    print(f"Container {container_name} is not running.")


Container apollo_dev_weidong is not running.
